In [1]:
import torchvision
import torch
import torchvision.transforms as transforms
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
DATA_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier\Datasets\Augment"
CHECKPOINT_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier\Models\PyTorch ResNet18 Model"
ONNX_SAVE_PATH = r"C:\Users\offco\Documents\Dev_Projects\BookCoverClassifier\Models\Quantized Model\pytorch_resnet18.onnx"
NUM_CLASSES = 199
TRAIN_EPOCHS = 150

In [3]:
mean = [0.3882, 0.3525, 0.3215]
std = [0.3120, 0.2901, 0.2704]

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),
])

data = torchvision.datasets.ImageFolder(root=DATA_PATH, transform=transform)

train_size = int(0.85 * len(data))
test_size = len(data) - train_size
train_data, test_data = torch.utils.data.random_split(data, [train_size, test_size])

In [4]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

In [5]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [6]:
def set_device():
    if torch.cuda.is_available(): dev = 'cuda:0'
    else: dev = 'cpu'
    
    return torch.device(dev)

def evaluate_model(model, test_loader):
    model.eval()
    
    predicted_correctly_on_epoch = 0
    total = 0
    
    device = set_device()
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            total += labels.size(0)
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            predicted_correctly_on_epoch += (predicted == labels).sum().item()
    
    epoch_acc = 100.0 * (predicted_correctly_on_epoch / total)
    print(f"Testing dataset: {int(predicted_correctly_on_epoch)} out of {total} correct. Accuracy: {round(epoch_acc, 3)}")
    
    return epoch_acc

def save_checkpoint(model, epoch, optimizer, best_acc):
    state = {
        'epoch': epoch+1,
        'model': model.state_dict(),
        'best accuracy': best_acc,
        'optimizer': optimizer.state_dict()
    }
    
    torch.save(state, f'{CHECKPOINT_PATH}\model_best_checkpoint.pth.tar')

In [7]:
def train_nn(model, train_loader, test_loader, criterion, optimizer, n_epochs):
    device = set_device()
    best_acc = 0
    
    for epoch in tqdm(range(n_epochs)):
        print(f"Epoch: {epoch+1}")
        model.train()
        running_loss = 0.0
        running_correct = 0.0
        total = 0
        
        for data in train_loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            total += labels.size(0)
            
            optimizer.zero_grad()
            
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            loss = criterion(outputs, labels)
            loss.backward()
            
            optimizer.step()
            
            running_loss += loss.item()
            running_correct += (labels == predicted).sum().item()
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100.00 * (running_correct / total)
        
        print(f"Training dataset: {int(running_correct)} out of {total} correct. Accuracy: {round(epoch_acc, 3)}. Loss: {round(epoch_loss, 3)}")
        
        test_acc = evaluate_model(model, test_loader)
        
        if (test_acc > best_acc):
            best_acc = test_acc
            save_checkpoint(model, epoch, optimizer, best_acc)
        
    print("Finished")
    return model

In [8]:
resnet18 = models.resnet18(pretrained=False)
num_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_features, NUM_CLASSES)

device = set_device()
resnet18 = resnet18.to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.parameters(), lr=0.01, momentum=0.9, weight_decay=0.003)

In [9]:
train_nn(resnet18, train_loader, test_loader, loss_func, optimizer, n_epochs=TRAIN_EPOCHS)

  0%|                                                                                          | 0/150 [00:00<?, ?it/s]

Epoch: 1
Training dataset: 409349 out of 1493921 correct. Accuracy: 27.401. Loss: 3.527


  1%|▌                                                                           | 1/150 [43:27<107:54:44, 2607.28s/it]

Testing dataset: 102862 out of 263634 correct. Accuracy: 39.017
Epoch: 2
Training dataset: 753682 out of 1493921 correct. Accuracy: 50.45. Loss: 2.285


  1%|▉                                                                         | 2/150 [1:26:32<106:39:33, 2594.41s/it]

...

Testing dataset: 147922 out of 263634 correct. Accuracy: 56.109
Epoch: 33
Training dataset: 937433 out of 1493921 correct. Accuracy: 62.75. Loss: 1.72


 22%|████████████████                                                         | 33/150 [23:39:23<83:49:19, 2579.14s/it]

Testing dataset: 148123 out of 263634 correct. Accuracy: 56.185
Epoch: 34


 22%|████████████████                                                         | 33/150 [24:02:34<85:14:34, 2622.86s/it]


KeyboardInterrupt: 

In [10]:
checkpoint = torch.load(f'{CHECKPOINT_PATH}\model_best_checkpoint.pth.tar')
print(f"Best Epoch: {checkpoint['epoch']}, Best Accuracy: {round(checkpoint['best accuracy'], 3)}")

Best Epoch: 13, Best Accuracy: 60.07


In [11]:
resnet18 = models.resnet18(pretrained=False)
num_features = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_features, NUM_CLASSES)
resnet18.load_state_dict(checkpoint['model'])

dummy_input = torch.randn(1, 3, 256, 256, dtype=torch.float32)
torch.onnx.export(
    resnet18, 
    dummy_input, 
    ONNX_SAVE_PATH,
    verbose=False,
    export_params=True,
    opset_version=11,
    input_names = ['image'],
    output_names = ['pred']
)